In [1]:
from datetime import datetime
import math
import time
import tensorflow as tf
import os

/home/hhl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
def conv_op(input_op, name, kh, kw, n_out, dh, dw, p):
    n_in = input_op.get_shape()[-1].value
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(scope + 'w', shape=[kh, kw, n_in, n_out], dtype=tf.float32, 
                                 initializer=tf.contrib.layers.xavier_initializer_conv2d())
        conv = tf.nn.conv2d(input_op, kernel, (1, dh, dw, 1), padding='SAME')
        bias_init_val = tf.constant(0.0, shape=[n_out], dtype=tf.float32)
        biases = tf.Variable(bias_init_val, trainable=True, name='b')
        z = tf.nn.bias_add(conv, biases)
        activation = tf.nn.relu(z, name=scope)
        p += [kernel,biases]
        return activation

In [4]:
def fc_op(input_op, name, n_out, p):
    n_in = input_op.get_shape()[-1].value
    with tf.name_scope(name) as scope:
        kernel = tf.get_variable(scope + 'w', shape=[n_in,n_out], dtype=tf.float32, 
                                 initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.Variable(tf.constant(0.1, shape=[n_out], dtype=tf.float32, name='b'))
        activation = tf.nn.relu_layer(input_op, kernel, biases, name=scope)
        p += [kernel, biases]
        return activation

In [5]:
def mpool_op(input_op, name, kh, kw, dh, dw):
    return tf.nn.max_pool(input_op, ksize=[1, kh, kw, 1], strides=[1, dh, dw, 1], padding='SAME', name=name)

In [6]:
def inference_op(input_op, keep_prob):
    p = []
    
    # 创建第一段卷积网络 -- outputs 112x112x64
    # 两个卷积层的卷积核都是3*3，卷积核数量（输出通道数）均为64，步长1*1，全像素扫描。
    conv1_1 = conv_op(input_op, name="conv1_1", kh=3, kw=3, n_out=64, dh=1, dw=1, p=p) # outputs 224x224x64
    conv1_2 = conv_op(conv1_1,  name="conv1_2", kh=3, kw=3, n_out=64, dh=1, dw=1, p=p) # outputs 224x224x64
    pool1 = mpool_op(conv1_2,   name="pool1",   kh=2, kw=2, dw=2, dh=2) # 标准的2*2的最大池化-outputs 112x112x64
    
    
    # 创建第二段卷积网络 -- outputs 56x56x128
    conv2_1 = conv_op(pool1,    name="conv2_1", kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)
    conv2_2 = conv_op(conv2_1,  name="conv2_2", kh=3, kw=3, n_out=128, dh=1, dw=1, p=p)
    pool2 = mpool_op(conv2_2,   name="pool2",   kh=2, kw=2, dh=2, dw=2)

    # 创建第三段卷积网络 -- outputs 28x28x256
    conv3_1 = conv_op(pool2,    name="conv3_1", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
    conv3_2 = conv_op(conv3_1,  name="conv3_2", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)
    conv3_3 = conv_op(conv3_2,  name="conv3_3", kh=3, kw=3, n_out=256, dh=1, dw=1, p=p)    
    pool3 = mpool_op(conv3_3,   name="pool3",   kh=2, kw=2, dh=2, dw=2)

    # 创建第四段卷积网络 -- outputs 14x14x512，此处写错一个,在copy 的时候将conv4_2 写成了conv3_2。
    # 由于tensorflow是静态计算图，没创建图的一个节点都要运行一下 前向和后向的计算过程，防止图构造出错。
    conv4_1 = conv_op(pool3, name='conv4_1', kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    conv4_2 = conv_op(conv4_1, name='conv4_2',kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    conv4_3 = conv_op(conv4_2, name='conv4_3',kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    pool4 = mpool_op(conv4_3, name='pool4',kh=2, kw=2, dh=2, dw=2)

    conv5_1 = conv_op(pool4, name='conv5_1', kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    conv5_2 = conv_op(conv5_1, name='conv5_2',kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    conv5_3 = conv_op(conv5_2, name='conv5_3',kh=3, kw=3, n_out=512, dh=1, dw=1, p=p)
    pool5 = mpool_op(conv5_3, name='pool5',kh=2, kw=2, dh=3, dw=2)
    
    shp = pool5.get_shape()
    flattened_shape = shp[1].value*shp[2].value*shp[3].value
    resh1 = tf.reshape(pool5,[-1,flattened_shape], name='resh1')
    
    fc6 = fc_op(resh1, name='fc6', n_out=4096, p=p)
    fc6_drop = tf.nn.dropout(fc6, keep_prob, name='fc6_drop')
    
    fc7 = fc_op(fc6_drop, name='fc7', n_out=4096, p=p)
    fc7_drop = tf.nn.dropout(fc7, keep_prob, name='fc7_drop')
    
    fc8 = fc_op(fc7_drop, name='fc8', n_out=1000, p=p)
    sotfmax = tf.nn.softmax(fc8)
    predictions = tf.argmax(sotfmax,1)
    return predictions, sotfmax, fc8, p

In [7]:
def time_tensorflow_run(session, target, feed, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    print(info_string)
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target, feed_dict=feed)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10 :
                print('%s: step%d, duration = %.3f' % (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration;
            total_duration_squared += duration*duration
    mn = total_duration / num_batches
    vr = total_duration_squared /num_batches - mn*mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec /batch' % (datetime.now(),info_string,num_batches,mn,sd))

In [8]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images = tf.Variable(tf.random_normal([batch_size,image_size,image_size,3], dtype=tf.float32, stddev=1e-1))
        keep_prob = tf.placeholder(tf.float32)
        predictions, sotfmax, fc8, p = inference_op(images, keep_prob)
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
#         time_tensorflow_run(sess, predictions, {keep_prob:1.0},"Forward")
        objective = tf.nn.l2_loss(fc8)
        grad = tf.gradients(objective, p)
        time_tensorflow_run(sess, grad, {keep_prob:0.5}, "Forward_backward")

In [9]:
batch_size = 32
num_batches = 100
run_benchmark()

Forward_backward
2018-11-11 16:47:48.377820: step0, duration = 0.266
2018-11-11 16:47:51.024765: step10, duration = 0.265
2018-11-11 16:47:53.673215: step20, duration = 0.265
2018-11-11 16:47:56.316280: step30, duration = 0.263
2018-11-11 16:47:58.956121: step40, duration = 0.266
2018-11-11 16:48:01.603988: step50, duration = 0.266
2018-11-11 16:48:04.252235: step60, duration = 0.265
2018-11-11 16:48:06.904079: step70, duration = 0.265
2018-11-11 16:48:09.551347: step80, duration = 0.264
2018-11-11 16:48:12.199909: step90, duration = 0.265
2018-11-11 16:48:14.582661: Forward_backward across 100 steps, 0.265 +/- 0.001 sec /batch
